# The challenge of future energy systems: Transmission Lines

### Authors: *Gonzalo Sánchez Contreras, Emilio de las Heras Páez de la Cadena, Ignacio Sanz Soriano*
### Date: *26/04/2017*

**Objective:** The aim of this notebook is to analyze and compare transmission line simulation results with the real measures obtained in the laboratory in different load scenarios to study the effects on voltage, current and losses. To achieve these goal, we use a scale transmission line to take measures that are then rescaled to real ones (132 kV, 100 MVA) and compare with simulation results using distributed parameters model.

## Technology (old and new) & Main Characteristics Low Voltage, Medium Voltage, High Voltage

![des](des.jpg "Title"){ width=50% }

<img src="C:\Users\nacho\Desktop\descarga.jpg">

#### Transmission Lines (High-Medium Voltage)
Transmission lines are those responsible for bulk movement of electrical energy (from generating substation to some consumption but mainly a distribution substation). Mostly AC systems, mix between Meshed and Radial.
-  HV(defined differently for different regions) is used to have lower losses (due to lower currents).
- Transmission lines are predominantly overhead lines (exceptions are underground lines used for urban areas or environmentally sensitive locations). 
- Transmission is mainly 3-Phase AC although 1-Phase AC is sometimes used.
- HVDC is sometimes implemented for greater efficiency over very long distances (hundreds of kms), also submarine power cables and lastly, in the interchange of power between grids that are not mutually synchronized.


#### Historically ... 
- In very early days, no transmission (DC & generation and loads located near each other).  19th 20th centuries saw a rapid industrialization of this power transmission system.

#### What the future holds ... 
- High-temperature superconductors (HTS) promise to revolutionize power distribution by providing lossless transmission of electrical power. This would translate in having a “supergrid”. This is a system based on an idea for combining very long electric power transmission with liquid nitrogen distribution, to achieve superconductivity in the power lines.
- Distributed generation presents a need for a two-directional distribution system: New regulation, smartgrids, storage (batteries, EVs), Voltage regulation capable of dealing with under and over voltage. 
- DC (IF costs severely drop)

## Simulation: Voltage increase & checking different values of impedance in the lines

#### Scale Transmission Line (Laboratory)
**Electric Patrameters**
Frequency = 50 Hz
Voltage = 220 V
Current = 5 A

$$ R = 1\:\Omega,\quad L = 11 mH,\quad C = 2\cdot 10^{-6}\:\mu F$$

**Measures taken in the lab, feeding a variable load ($cos(\varphi)$)**

In [2]:
import numpy as np

V_inicial = [ 222, 221, 221, 221, 222, 222 ]
V_final = [ 224, 216, 208, 199, 190, 182 ]
Pot = [ 0, 1000, 2000, 3000, 4000, 5000]
print(Pot[1])

1000


Now, we calculate the $R_{load}$ for each power consumption.
$$R_{load}= \frac{(U_{load}^{(i)})^2}{P_{load}^{(i)}}$$

In [3]:
# Resistencia de carga
x = range(0,6)
R_load = []

v = []
for i in x:
    if Pot[i] == 0:
        R_load.append(np.inf)
    else:
        R_load.append(V_final[i]**2/Pot[i])
        
print(R_load)

[inf, 46.656, 21.632, 13.200333333333333, 9.025, 6.6248]


### Base magnitudes of the Line to calculate *per unit* measures


In [4]:
frecuencia = 50
omega = 2+np.pi*frecuencia

# Magnitudes base de la línea
U = 220
I = 5
S = np.sqrt(3)*U*I
Z = U**2/S

# Parámetros eléctricos de la línea
R = 1
L = 11e-3
C = 2e-6

# Parámetros en p.u.
r = R/Z
x = omega*L/Z
b = omega*C*Z

r_load = R_load/Z

# Medidas en p.u.
v_inicial = [x/U for x in V_inicial]
v_final = [x/U for x in V_final]

### Voltage Drop ($\Delta V$) vs. Power Cosumption ($P_{load}$)

In [5]:
deltaV = [v_inicial[i] - v_final[i] for i in range(len(v_inicial))]
print(deltaV)

[-0.009090909090909038, 0.022727272727272818, 0.059090909090909194, 0.10000000000000009, 0.1454545454545454, 0.18181818181818177]


### Plotting Voltage Drop 

In [14]:
import matplotlib.pyplot as plt


In [15]:
plt.plot(Pot,deltaV)

RuntimeError: LaTeX was not able to process the following string:
b'lp'
Here is the full report generated by LaTeX: 

